In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz


In [ ]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz


In [ ]:
!pip install -q findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.0.0-bin-hadoop3.2/python/pyspark'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Creating Data FRame from an RDD

In [ ]:
columns = ["language", "users"]
data = [("java", "20000"), ("python", "30000"), ("scala", "40000")]
rdd = spark.sparkContext.parallelize(data)
rdd.collect()

[('java', '20000'), ('python', '30000'), ('scala', '40000')]

PySpark RDD’s toDF() method is used to create a DataFrame from existing RDD. Since RDD doesn’t have columns, the DataFrame is created with default column names “_1” and “_2” as we have two columns.



In [ ]:
dfFromRDD1 = rdd.toDF()
dfFromRDD1.printSchema()


root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)



In [ ]:
dfFromRDD1.show()

+------+-----+
|    _1|   _2|
+------+-----+
|  java|20000|
|python|30000|
| scala|40000|
+------+-----+



**Create Data Frame with schema**

If you wanted to specify the column names along with their data types, you should create the StructType schema first and then assign this while creating a DataFrame.



In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
data1 = [("Smith", "Jone", "123", "M", 20000),
         ("Nance", "watch", "456", "F", 10000),
         ("Rani", "Singh", "234", "F", 30000),
         ("Raju", "Thomas", "897", "M", 15000),
         ("Ranjan", "Sharma", "862", "M", 25000)
]

schema = StructType([ \
    StructField("FirstName", StringType(), True), \
    StructField("LastName", StringType(), True), \
    StructField("Id", StringType(), True), \
    StructField("Gender", StringType(), True), \
    StructField("Salary", IntegerType(), True) \
])

df = spark.createDataFrame(data=data1, schema =schema)
df.printSchema()
df.show()
        


root
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Salary: integer (nullable = true)

+---------+--------+---+------+------+
|FirstName|LastName| Id|Gender|Salary|
+---------+--------+---+------+------+
|    Smith|    Jone|123|     M| 20000|
|    Nance|   watch|456|     F| 10000|
|     Rani|   Singh|234|     F| 30000|
|     Raju|  Thomas|897|     M| 15000|
|   Ranjan|  Sharma|862|     M| 25000|
+---------+--------+---+------+------+



In [ ]:
pandasDF = df.toPandas()
print(pandasDF)

  FirstName LastName   Id Gender  Salary
0     Smith     Jone  123      M   20000
1     Nance    watch  456      F   10000
2      Rani    Singh  234      F   30000
3      Raju   Thomas  897      M   15000
4    Ranjan   Sharma  862      M   25000


Creating Data FRame from Text File

In [ ]:
df1 = spark.read.text("/content/TextFile.txt")

In [ ]:
df1.show(2,truncate=False,vertical=True)

-RECORD 0---------------------------------
 value | Project Gutenberg’s              
-RECORD 1---------------------------------
 value | Alice’s Adventures in Wonderland 
only showing top 2 rows



# PySpark Row

It's basically a record/row in a Data Frame

It can be imported using : import pyspark.sql.Row

**Creating** **a** **Row** **object**

In [ ]:
from pyspark.sql import Row
row=Row("james", 40)
print(row[0],","+str(row[1]))

james ,40


In [ ]:
row=Row(name="Alice", age=11)
print(row)
print(row.name)

Row(name='Alice', age=11)
Alice


# Creaing a custom class from Row

We can create a class and access it just like objects

In [ ]:
from pyspark.sql import Row
Person = Row("Name", "Age")
p1 = Person("Jame", 40)
p2 = Person("Alice", 35)
print(p1.Name + ","+p2.Name)


Jame,Alice


Using Row class on PySpark RDD

In [ ]:
from pyspark.sql import SparkSession, Row
spark = SparkSession.builder.appName("abc.com").getOrCreate()

data = [Row(name = "James, Smith", lang = ["java", "python", "c"], state = "CA"),
        Row(name = "Michel, Rose", lang = ["python", "c++", "c"], state = "NJ"),
        Row(name = "Robert, Williams", lang = ["scala", "java", "python"], state = "NV")]
rdd = spark.sparkContext.parallelize(data)
rdd.collect()

[Row(name='James, Smith', lang=['java', 'python', 'c'], state='CA'),
 Row(name='Michel, Rose', lang=['python', 'c++', 'c'], state='NJ'),
 Row(name='Robert, Williams', lang=['scala', 'java', 'python'], state='NV')]

Now lets collect this data and access it using its properties

In [ ]:

collData=rdd.collect()
for row in collData:
    print(row.name + "," +str(row.lang)+ "," +str(row.state))


James, Smith,['java', 'python', 'c'],CA
Michel, Rose,['python', 'c++', 'c'],NJ
Robert, Williams,['scala', 'java', 'python'],NV


**Using** **a** **Row** **class** **on** **Data** **Frames**

In [ ]:
df = spark.createDataFrame(data)
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- lang: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: string (nullable = true)

+----------------+--------------------+-----+
|            name|                lang|state|
+----------------+--------------------+-----+
|    James, Smith|   [java, python, c]|   CA|
|    Michel, Rose|    [python, c++, c]|   NJ|
|Robert, Williams|[scala, java, pyt...|   NV|
+----------------+--------------------+-----+



Changing column names using `toDF()` function

In [ ]:
columns = ["FullName", "Language", "State"]
df = spark.createDataFrame(data).toDF(*columns)
df.printSchema()
df.show()

root
 |-- FullName: string (nullable = true)
 |-- Language: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- State: string (nullable = true)

+----------------+--------------------+-----+
|        FullName|            Language|State|
+----------------+--------------------+-----+
|    James, Smith|   [java, python, c]|   CA|
|    Michel, Rose|    [python, c++, c]|   NJ|
|Robert, Williams|[scala, java, pyt...|   NV|
+----------------+--------------------+-----+



# PySpark Column Class


*   It is used to manipulate column values
*   To evaluate boolean expressions to filter rows
*   To retrieve a value or a part of a value from a Data Frame
*   To work with list, map & struct columns.





Creating a column class object

In [ ]:
from pyspark.sql.functions import lit
colObj = ("abc.com")

data = [("James", 22), ("Ann", 24)]
df = spark.createDataFrame(data).toDF("Name", "Age")
df.printSchema()

df.select(df.Name).show()
df.select(df.Age).show()

from pyspark.sql.functions import col
df.select(col("Name")).show()
df.select(col("Age")).show()
df.show()


root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)

+-----+
| Name|
+-----+
|James|
|  Ann|
+-----+

+---+
|Age|
+---+
| 22|
| 24|
+---+

+-----+
| Name|
+-----+
|James|
|  Ann|
+-----+

+---+
|Age|
+---+
| 22|
| 24|
+---+

+-----+---+
| Name|Age|
+-----+---+
|James| 22|
|  Ann| 24|
+-----+---+



**PySpark Column Operators**

Allows us to do Arithematic operations on columns using operators.

In [ ]:
data = [(100,2,1), (200,3,4), (300,4,4)]
df = spark.createDataFrame(data).toDF("col1", "col2", "col3")

df.select(df.col1 + df.col2).show()
df.select(df.col1 - df.col2).show()
df.select(df.col1 * df.col2).show()
df.select(df.col1 / df.col2).show()
df.select(df.col1 % df.col2).show()

df.select(df.col1 > df.col2).show()
df.select(df.col1 < df.col2).show()
df.select(df.col1 == df.col2).show()

+-------------+
|(col1 + col2)|
+-------------+
|          102|
|          203|
|          304|
+-------------+

+-------------+
|(col1 - col2)|
+-------------+
|           98|
|          197|
|          296|
+-------------+

+-------------+
|(col1 * col2)|
+-------------+
|          200|
|          600|
|         1200|
+-------------+

+-----------------+
|    (col1 / col2)|
+-----------------+
|             50.0|
|66.66666666666667|
|             75.0|
+-----------------+

+-------------+
|(col1 % col2)|
+-------------+
|            0|
|            2|
|            0|
+-------------+

+-------------+
|(col1 > col2)|
+-------------+
|         true|
|         true|
|         true|
+-------------+

+-------------+
|(col1 < col2)|
+-------------+
|        false|
|        false|
|        false|
+-------------+

+-------------+
|(col1 = col2)|
+-------------+
|        false|
|        false|
|        false|
+-------------+



**PySpark WithColumnRenamed() to rename a column**

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
data = [("Smith", "Jone", "123", "M", 20000),
         ("Nance", "watch", "456", "F", 10000),
         ("Rani", "Singh", "234", "F", 30000),
         ("Raju", "Thomas", "897", "M", 15000),
         ("Ranjan", "Sharma", "862", "M", 25000)
]

schema = StructType([ \
    StructField("FirstName", StringType(), True), \
    StructField("LastName", StringType(), True), \
    StructField("Id", StringType(), True), \
    StructField("Gender", StringType(), True), \
    StructField("Salary", IntegerType(), True) \
])

df = spark.createDataFrame(data=data, schema =schema)
df.printSchema()
df.show()
        

root
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Salary: integer (nullable = true)

+---------+--------+---+------+------+
|FirstName|LastName| Id|Gender|Salary|
+---------+--------+---+------+------+
|    Smith|    Jone|123|     M| 20000|
|    Nance|   watch|456|     F| 10000|
|     Rani|   Singh|234|     F| 30000|
|     Raju|  Thomas|897|     M| 15000|
|   Ranjan|  Sharma|862|     M| 25000|
+---------+--------+---+------+------+



In [ ]:
df.withColumnRenamed("FirstName", "fname").show()

+------+--------+---+------+------+
| fname|LastName| Id|Gender|Salary|
+------+--------+---+------+------+
| Smith|    Jone|123|     M| 20000|
| Nance|   watch|456|     F| 10000|
|  Rani|   Singh|234|     F| 30000|
|  Raju|  Thomas|897|     M| 15000|
|Ranjan|  Sharma|862|     M| 25000|
+------+--------+---+------+------+



In [ ]:
df1 = df.withColumnRenamed("LasName", "lname") \
      .withColumnRenamed("Gender", "Sex")
df1.printSchema()
df1.show()

root
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Salary: integer (nullable = true)

+---------+--------+---+---+------+
|FirstName|LastName| Id|Sex|Salary|
+---------+--------+---+---+------+
|    Smith|    Jone|123|  M| 20000|
|    Nance|   watch|456|  F| 10000|
|     Rani|   Singh|234|  F| 30000|
|     Raju|  Thomas|897|  M| 15000|
|   Ranjan|  Sharma|862|  M| 25000|
+---------+--------+---+---+------+



To Rename Multiple Columns

In [ ]:
df2 = df1.withColumnRenamed("FirstName", "fname") \
      .withColumnRenamed("LastName", "lname")

df2.show()

+------+------+---+---+------+
| fname| lname| Id|Sex|Salary|
+------+------+---+---+------+
| Smith|  Jone|123|  M| 20000|
| Nance| watch|456|  F| 10000|
|  Rani| Singh|234|  F| 30000|
|  Raju|Thomas|897|  M| 15000|
|Ranjan|Sharma|862|  M| 25000|
+------+------+---+---+------+



Use withColumn()


*   To change data type of a column
*   To update the value of an existing column
*   To create a column from an existing column
*   To add a new column
*   To rename a column
*   To drop a column



Changing Data Type of a column

In [ ]:
df2.withColumn("Salary", col("Salary").cast("Float")).show()

+------+------+---+---+-------+
| fname| lname| Id|Sex| Salary|
+------+------+---+---+-------+
| Smith|  Jone|123|  M|20000.0|
| Nance| watch|456|  F|10000.0|
|  Rani| Singh|234|  F|30000.0|
|  Raju|Thomas|897|  M|15000.0|
|Ranjan|Sharma|862|  M|25000.0|
+------+------+---+---+-------+



Updating the value of an existig column

In [ ]:
df2.withColumn("Salary", col("Salary")*100).show()

+------+------+---+---+-------+
| fname| lname| Id|Sex| Salary|
+------+------+---+---+-------+
| Smith|  Jone|123|  M|2000000|
| Nance| watch|456|  F|1000000|
|  Rani| Singh|234|  F|3000000|
|  Raju|Thomas|897|  M|1500000|
|Ranjan|Sharma|862|  M|2500000|
+------+------+---+---+-------+



Creating column from an existing column

In [ ]:
df2.withColumn("UpdatedSalary", col("Salary")*1).show()

+------+------+---+---+------+-------------+
| fname| lname| Id|Sex|Salary|UpdatedSalary|
+------+------+---+---+------+-------------+
| Smith|  Jone|123|  M| 20000|        20000|
| Nance| watch|456|  F| 10000|        10000|
|  Rani| Singh|234|  F| 30000|        30000|
|  Raju|Thomas|897|  M| 15000|        15000|
|Ranjan|Sharma|862|  M| 25000|        25000|
+------+------+---+---+------+-------------+



Adding a new column using withColumn() and lit

In [ ]:
df2.withColumn("Country", lit("USA")).show()

+------+------+---+---+------+-------+
| fname| lname| Id|Sex|Salary|Country|
+------+------+---+---+------+-------+
| Smith|  Jone|123|  M| 20000|    USA|
| Nance| watch|456|  F| 10000|    USA|
|  Rani| Singh|234|  F| 30000|    USA|
|  Raju|Thomas|897|  M| 15000|    USA|
|Ranjan|Sharma|862|  M| 25000|    USA|
+------+------+---+---+------+-------+



In [ ]:
df2.withColumnRenamed("Sex", "Gender").show()

+------+------+---+------+------+
| fname| lname| Id|Gender|Salary|
+------+------+---+------+------+
| Smith|  Jone|123|     M| 20000|
| Nance| watch|456|     F| 10000|
|  Rani| Singh|234|     F| 30000|
|  Raju|Thomas|897|     M| 15000|
|Ranjan|Sharma|862|     M| 25000|
+------+------+---+------+------+



In [ ]:
df2.withColumn("UpdatedSalary", col("Salary") *1).show()

+------+------+---+---+------+-------------+
| fname| lname| Id|Sex|Salary|UpdatedSalary|
+------+------+---+---+------+-------------+
| Smith|  Jone|123|  M| 20000|        20000|
| Nance| watch|456|  F| 10000|        10000|
|  Rani| Singh|234|  F| 30000|        30000|
|  Raju|Thomas|897|  M| 15000|        15000|
|Ranjan|Sharma|862|  M| 25000|        25000|
+------+------+---+---+------+-------------+



In [ ]:
df2.drop("UpdatedSalary").show()

+------+------+---+---+------+
| fname| lname| Id|Sex|Salary|
+------+------+---+---+------+
| Smith|  Jone|123|  M| 20000|
| Nance| watch|456|  F| 10000|
|  Rani| Singh|234|  F| 30000|
|  Raju|Thomas|897|  M| 15000|
|Ranjan|Sharma|862|  M| 25000|
+------+------+---+---+------+



In [67]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, ArrayType

data2 = [("Aruna", ["Java", "Scala", "Python"], "India", "F"),
         ("Smith", ["Pyspark", "C#", "Java"], "USA", "M"),
         ("Jacob", ["Java", "C", "Javascript"], "USA", "M"),
         ("chenchu", ["Scala", "C", "Python"], "China", "M"),
         ("Veena", ["C++", "Java", "Python"], "India", "F")
         ]
schema = StructType([
    StructField("Name", StringType(), True), 
    StructField("Languages", StringType(), True), 
    StructField("Country", StringType(), True), 
    StructField("Gender", StringType(), True) 
])

df = spark.createDataFrame(data=data2, schema=schema)
df.printSchema()
df.show(truncate=False)


root
 |-- Name: string (nullable = true)
 |-- Languages: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)

+-------+---------------------+-------+------+
|Name   |Languages            |Country|Gender|
+-------+---------------------+-------+------+
|Aruna  |[Java, Scala, Python]|India  |F     |
|Smith  |[Pyspark, C#, Java]  |USA    |M     |
|Jacob  |[Java, C, Javascript]|USA    |M     |
|chenchu|[Scala, C, Python]   |China  |M     |
|Veena  |[C++, Java, Python]  |India  |F     |
+-------+---------------------+-------+------+



**PySpark Where Filter Function | Multiple Conditions**

DataFrame filter() with column conditions 

In [68]:
df.filter(df.Country == "India").show(truncate=False)

+-----+---------------------+-------+------+
|Name |Languages            |Country|Gender|
+-----+---------------------+-------+------+
|Aruna|[Java, Scala, Python]|India  |F     |
|Veena|[C++, Java, Python]  |India  |F     |
+-----+---------------------+-------+------+



In [69]:
df.filter(df.Gender != "M").show(truncate=False)

+-----+---------------------+-------+------+
|Name |Languages            |Country|Gender|
+-----+---------------------+-------+------+
|Aruna|[Java, Scala, Python]|India  |F     |
|Veena|[C++, Java, Python]  |India  |F     |
+-----+---------------------+-------+------+



**PySpark filter with multiple conditions**

In [74]:
df.filter((df.Gender == "M") & (df.Country == "USA")).show(truncate=False)

+-----+---------------------+-------+------+
|Name |Languages            |Country|Gender|
+-----+---------------------+-------+------+
|Smith|[Pyspark, C#, Java]  |USA    |M     |
|Jacob|[Java, C, Javascript]|USA    |M     |
+-----+---------------------+-------+------+



**Filter based on List values**

In [76]:
li = ["China", "India"]
df.filter(df.Country.isin(li)).show()
df.filter(~df.Country.isin(li)).show()

+-------+--------------------+-------+------+
|   Name|           Languages|Country|Gender|
+-------+--------------------+-------+------+
|  Aruna|[Java, Scala, Pyt...|  India|     F|
|chenchu|  [Scala, C, Python]|  China|     M|
|  Veena| [C++, Java, Python]|  India|     F|
+-------+--------------------+-------+------+

+-----+--------------------+-------+------+
| Name|           Languages|Country|Gender|
+-----+--------------------+-------+------+
|Smith| [Pyspark, C#, Java]|    USA|     M|
|Jacob|[Java, C, Javascr...|    USA|     M|
+-----+--------------------+-------+------+



**Filter based on startsWith(), endsWith() & contains()**

In [77]:
df.filter(df.Country.startswith("I")).show()
df.filter(df.Country.endswith("a")).show()
df.filter(df.Gender.contains("M")).show()

+-----+--------------------+-------+------+
| Name|           Languages|Country|Gender|
+-----+--------------------+-------+------+
|Aruna|[Java, Scala, Pyt...|  India|     F|
|Veena| [C++, Java, Python]|  India|     F|
+-----+--------------------+-------+------+

+-------+--------------------+-------+------+
|   Name|           Languages|Country|Gender|
+-------+--------------------+-------+------+
|  Aruna|[Java, Scala, Pyt...|  India|     F|
|chenchu|  [Scala, C, Python]|  China|     M|
|  Veena| [C++, Java, Python]|  India|     F|
+-------+--------------------+-------+------+

+-------+--------------------+-------+------+
|   Name|           Languages|Country|Gender|
+-------+--------------------+-------+------+
|  Smith| [Pyspark, C#, Java]|    USA|     M|
|  Jacob|[Java, C, Javascr...|    USA|     M|
|chenchu|  [Scala, C, Python]|  China|     M|
+-------+--------------------+-------+------+



**PySpark Discting to Drop Duplicates**



`distinct()` function is used to drop the duplicate rows from the data frame whereas `dropDuplicates()` is used to doprows based on selected columns.

In [82]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
data3 = [("James", "Sales", 3000 ),
         ("Rani", "Marketing", 4000),
         ("Jacob", "Sales", 3500),
         ("Mathew", "Finance", 5000),
         ("Maria", "Sales", 4500),
         ("Jane", "Marketing", 4500),
         ("Kumar", "Finance", 5500),
         ("Saif", "Sales", 6000),
         ("Sonu", "Finance", 6400),
         ("Reema", "Sales", 7000)]

columns = ["Name", "Department", "Salary"]
df = spark.createDataFrame(data = data3, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)

+------+----------+------+
|Name  |Department|Salary|
+------+----------+------+
|James |Sales     |3000  |
|Rani  |Marketing |4000  |
|Jacob |Sales     |3500  |
|Mathew|Finance   |5000  |
|Maria |Sales     |4500  |
|Jane  |Marketing |4500  |
|Kumar |Finance   |5500  |
|Saif  |Sales     |6000  |
|Sonu  |Finance   |6400  |
|Reema |Sales     |7000  |
+------+----------+------+



To get Distinct rows

In [83]:
distinctDF = df.distinct()
distinctDF.show(truncate=False)

+------+----------+------+
|Name  |Department|Salary|
+------+----------+------+
|Mathew|Finance   |5000  |
|Maria |Sales     |4500  |
|Sonu  |Finance   |6400  |
|Saif  |Sales     |6000  |
|James |Sales     |3000  |
|Reema |Sales     |7000  |
|Rani  |Marketing |4000  |
|Jacob |Sales     |3500  |
|Kumar |Finance   |5500  |
|Jane  |Marketing |4500  |
+------+----------+------+



To eliminate duplicate entries from multiple columns

In [84]:
dropDisDF = df.dropDuplicates(["Department", "Salary"])
dropDisDF.show(truncate=False)

+------+----------+------+
|Name  |Department|Salary|
+------+----------+------+
|Sonu  |Finance   |6400  |
|Saif  |Sales     |6000  |
|Rani  |Marketing |4000  |
|Jane  |Marketing |4500  |
|Jacob |Sales     |3500  |
|Mathew|Finance   |5000  |
|Maria |Sales     |4500  |
|James |Sales     |3000  |
|Reema |Sales     |7000  |
|Kumar |Finance   |5500  |
+------+----------+------+



`orderBy()` & `sort()` to arrange the columns in ascending or descending order on single or multiple functions.

In [85]:
df.sort("Salary").show(truncate=False)

+------+----------+------+
|Name  |Department|Salary|
+------+----------+------+
|James |Sales     |3000  |
|Jacob |Sales     |3500  |
|Rani  |Marketing |4000  |
|Maria |Sales     |4500  |
|Jane  |Marketing |4500  |
|Mathew|Finance   |5000  |
|Kumar |Finance   |5500  |
|Saif  |Sales     |6000  |
|Sonu  |Finance   |6400  |
|Reema |Sales     |7000  |
+------+----------+------+



In [86]:
df.orderBy("Name", "Department", "Salary").show(truncate=False)

+------+----------+------+
|Name  |Department|Salary|
+------+----------+------+
|Jacob |Sales     |3500  |
|James |Sales     |3000  |
|Jane  |Marketing |4500  |
|Kumar |Finance   |5500  |
|Maria |Sales     |4500  |
|Mathew|Finance   |5000  |
|Rani  |Marketing |4000  |
|Reema |Sales     |7000  |
|Saif  |Sales     |6000  |
|Sonu  |Finance   |6400  |
+------+----------+------+



In [87]:
df.orderBy(df.Department.asc(), df.Salary.desc(), df.Name.asc()).show()

+------+----------+------+
|  Name|Department|Salary|
+------+----------+------+
|  Sonu|   Finance|  6400|
| Kumar|   Finance|  5500|
|Mathew|   Finance|  5000|
|  Jane| Marketing|  4500|
|  Rani| Marketing|  4000|
| Reema|     Sales|  7000|
|  Saif|     Sales|  6000|
| Maria|     Sales|  4500|
| Jacob|     Sales|  3500|
| James|     Sales|  3000|
+------+----------+------+



groupBy() function:used to collect identical data into groups

It contains:
*  count()
*  mean()
*  max()
*  avg()
*  sum()
*  agg()
*  pivot()

In [88]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
data4 = [("James", "Sales", 3000, "USA" ),
         ("Rani", "Marketing", 4000, "Nepal"),
         ("Jacob", "Sales", 3500, "USA"),
         ("Mathew", "Finance", 5000, "USA"),
         ("Maria", "Sales", 4500, "Bangladesh"),
         ("Jane", "Marketing", 4500, "USA"),
         ("Kumar", "Finance", 5500, "India"),
         ("Saif", "Sales", 6000, "India"),
         ("Sonu", "Finance", 6400, "India"),
         ("Reema", "Sales", 7000, "Nepal")]

columns = ["Name", "Department", "Salary", "Country"]
df = spark.createDataFrame(data = data4, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Country: string (nullable = true)

+------+----------+------+----------+
|Name  |Department|Salary|Country   |
+------+----------+------+----------+
|James |Sales     |3000  |USA       |
|Rani  |Marketing |4000  |Nepal     |
|Jacob |Sales     |3500  |USA       |
|Mathew|Finance   |5000  |USA       |
|Maria |Sales     |4500  |Bangladesh|
|Jane  |Marketing |4500  |USA       |
|Kumar |Finance   |5500  |India     |
|Saif  |Sales     |6000  |India     |
|Sonu  |Finance   |6400  |India     |
|Reema |Sales     |7000  |Nepal     |
+------+----------+------+----------+



In [89]:
df.groupBy("Department").sum("Salary").show(truncate=False)

+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|Sales     |24000      |
|Finance   |16900      |
|Marketing |8500       |
+----------+-----------+



In [90]:
df.groupBy("Salary").count().show()

+------+-----+
|Salary|count|
+------+-----+
|  7000|    1|
|  5500|    1|
|  6400|    1|
|  4500|    2|
|  4000|    1|
|  6000|    1|
|  3500|    1|
|  3000|    1|
|  5000|    1|
+------+-----+



In [93]:
df.groupBy("Department").min("Salary").show(truncate=False)



+----------+-----------+
|Department|min(Salary)|
+----------+-----------+
|Sales     |3000       |
|Finance   |5000       |
|Marketing |4000       |
+----------+-----------+



In [94]:
df.groupBy("Department").mean("Salary").show(truncate=False)

+----------+-----------------+
|Department|avg(Salary)      |
+----------+-----------------+
|Sales     |4800.0           |
|Finance   |5633.333333333333|
|Marketing |4250.0           |
+----------+-----------------+



**groupBy() & aggregate functions on multiple columns**